# Домашнее задание 2
### Андреев Никита

In [3]:
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.optimize as opt
import sklearn.datasets
from sklearn.model_selection import train_test_split

#### Определим функцию $\sigma$:

## $\sigma_w(x)=\frac{e^{w^tx}}{1+e^{w^tx}}=\frac{1}{1+e^{-w^tx}}$

Тогда
$1-\sigma_w(x)=1-\frac{1}{1+e^{-w^tx}}=\frac{1}{1+e^{w^tx}}=\sigma_w(-x)$

#### Функция, которую необходимо минимизировать:
## $f(w)=-\frac{1}{N}\sum\limits_1^n\log(\sigma_w(y_i x_i))$, $y_i =\pm 1$

#### Посчитаем производные:

### $\nabla\sigma_w(x)=\frac{e^{-w^tx}}{(1+e^{-w^tx})^2}x = \sigma_w(x) \sigma_w(-x)x$

### $\nabla f(w) = -\frac{1}{N}\sum\limits_1^n\sigma_w(-y_i x_i)y_i x_i$

### $\nabla^2 f(w) = \frac{1}{N}\sum\limits_1^n\sigma_w(y_i x_i)\sigma_w(-y_i x_i)x_i x_i^t=
\frac{1}{N}\sum\limits_1^n\sigma_w(x_i)\sigma_w(-x_i)x_i x_i^t$

In [4]:
def sigma(w, x):
    return 1 / (1 + np.exp(-np.dot(w, x)))


def function(w, X, labels):
    return -1 / len(X) * np.sum([np.log(sigma(w, l * x)) for x, l in zip(X, labels)])


def gradient(w, X, labels):
    return -1 / len(X) * np.sum([l * x * sigma(w, -l * x) for x, l in zip(X, labels)], axis=0)


def hessian(w, X, labels):
    return 1 / len(X) * np.sum([np.outer(x, x) * sigma(w, x) * sigma(w, -x) for x in X], axis=0)


#### Тесты для проверки правильности вычисления производных:

In [6]:
#численное дифференцирование
def der(fun, point, epsilon=np.sqrt(sys.float_info.epsilon)):
    return (fun(point + epsilon) - fun(point)) / epsilon

#считает среднюю относительную погрешность по всем кординатным осям в рандомной точке
def check_gradient(fun, grad, R, dim, args=(), diff_eps=np.sqrt(sys.float_info.epsilon)):
    w = np.random.random(dim)
    w = (2 * w - 1) * R
    dw = np.eye(dim)
    difs = [np.abs((np.dot(grad(w, *args), dw_i) - der(lambda t: fun(w + t * dw_i, *args), 0, diff_eps)) / np.dot(
        grad(w, *args), dw_i)) for dw_i in dw]
    return np.average(difs)

#считает среднюю относительную погрешность по всем парам кординатных осей в рандомной точке
def check_hessian(grad, hess, R, dim, args=(), diff_eps=np.sqrt(sys.float_info.epsilon)):
    w = np.random.random(dim)
    w = (2 * w - 1) * R
    dw = np.eye(dim)
    g = hess(w, *args)

    def xAy(A, x, y):
        return np.dot(x, np.dot(A, y))

    difs = [np.abs((xAy(g, dw1, dw2) - der(lambda t: np.dot(grad(w + t * dw1, *args), dw2), 0, diff_eps))
                   / xAy(g, dw1, dw2)) for dw1 in dw for dw2 in dw]
    return np.average(difs)
